# <font color="orange"> House Prices - Advanced Regression Techniques </font>

#### Predict sales prices and practice feature engineering, RFs, and gradient boosting

In [117]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [118]:
DataFolder = "/Users/manideepbangaru/Documents/EDAnMLApply/Datasets/house-prices-advanced-regression-techniques/"
OutputFolder = "/Users/manideepbangaru/Documents/EDAnMLApply/Output"

In [121]:
hdf = pd.read_csv(DataFolder+"train.csv",index_col=0)
hdf.reset_index(drop=True, inplace=True)
hdf_y = hdf["SalePrice"]
hdf.drop(columns=["SalePrice"],inplace=True)

In [122]:
hdf.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


### Dealing with Missing Values

In [123]:
for cols in hdf.columns:
    nullsum = round((hdf[cols].isnull().sum()/len(hdf))*100,2)
    if nullsum >= 50 :
        hdf.drop(cols,axis=1,inplace=True)
        print("dropped column %s which has %s percentage of null values"%(cols,nullsum))

dropped column Alley which has 93.77 percentage of null values
dropped column PoolQC which has 99.52 percentage of null values
dropped column Fence which has 80.75 percentage of null values
dropped column MiscFeature which has 96.3 percentage of null values


In [124]:
hdf.reset_index(drop=True,inplace=True)
hdf

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [125]:
for idx in range(len(hdf)):
    nullsum = round((hdf.loc[idx].isnull().sum()/len(hdf.columns))*100,2)
    if nullsum > 50:
        hdf.drop(idx,axis=0,inplace=True)
        print("dropping row %s"%idx)

### Separating Numerical and Categorical data

In [126]:
hdf.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 75, dtype: object

In [127]:
hdf["MSSubClass"] = hdf["MSSubClass"].astype("object")
hdf.dtypes

MSSubClass        object
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 75, dtype: object

In [128]:
Num_Data = hdf[hdf.columns[hdf.dtypes != object]]
Cat_Data = hdf[hdf.columns[hdf.dtypes == object]]

In [129]:
len(Num_Data.columns) + len(Cat_Data.columns) == len(hdf.columns)

True

### Imputation

In [130]:
from sklearn.impute import KNNImputer

In [131]:
Num_Data.loc[:,Num_Data.isnull().any()].isnull().sum()/len(Num_Data)*100

LotFrontage    17.739726
MasVnrArea      0.547945
GarageYrBlt     5.547945
dtype: float64

In [132]:
imputer = KNNImputer(n_neighbors=3)

In [133]:
imputer.fit(Num_Data.values)

KNNImputer(n_neighbors=3)

In [134]:
Num_Data_Trans_values = imputer.transform(Num_Data.values)

In [135]:
Num_Data_Trans = pd.DataFrame(Num_Data_Trans_values,columns = Num_Data.columns)

To impute categorical variables, we need to first convert them to dummy variables

In [136]:
Cat_Data = pd.get_dummies(Cat_Data)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


### Outlier treatment

In [137]:
from scipy.stats import zscore

In [138]:
zscores = zscore(Num_Data_Trans)

In [139]:
abs_z_scores = np.abs(zscores)
filtered_entries = (abs_z_scores < 3).all(axis=1)

In [140]:
New_Num_Data = Num_Data[filtered_entries]

In [141]:
New_Num_Data.reset_index(drop=True,inplace=True)

### Standardization

In [142]:
from sklearn.preprocessing import StandardScaler

In [143]:
SS = StandardScaler()
Standardized_Num_Data = pd.DataFrame(SS.fit_transform(New_Num_Data),columns=New_Num_Data.columns)

### Preparing the data

In [144]:
df = pd.concat([Standardized_Num_Data,Cat_Data[filtered_entries].reset_index(drop=True,inplace=True)],axis=1)

In [157]:
y_filt = hdf_y[filtered_entries]

In [161]:
y_filt.index = range(len(y_filt))

### Train Test Split

In [163]:
from sklearn.model_selection import train_test_split

In [169]:
X_train,X_test,y_train,y_test = train_test_split(df,y_filt,test_size=0.3, random_state=123)

In [170]:
df.corr()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
LotFrontage,1.000000,0.676169,0.242716,-0.038225,0.152935,0.087849,0.148737,0.175844,-0.023832,0.151633,...,0.367536,0.135254,0.155702,-0.032054,-0.003976,0.040076,NaN,0.027223,0.013449,0.000238
LotArea,0.676169,1.000000,0.143141,-0.063250,0.061145,0.005068,0.089639,0.170325,-0.010906,0.082340,...,0.296627,0.164265,0.087027,-0.033980,0.008508,0.050167,NaN,0.038664,0.001714,-0.023102
OverallQual,0.242716,0.143141,1.000000,-0.193374,0.643484,0.582143,0.399781,0.171829,-0.142646,0.321134,...,0.572683,0.255763,0.370869,-0.173375,-0.001868,-0.034948,NaN,-0.061430,0.090131,-0.030673
OverallCond,-0.038225,-0.063250,-0.193374,1.000000,-0.423477,-0.027233,-0.138321,-0.008627,0.070226,-0.151016,...,-0.189139,-0.048219,-0.105569,0.130830,-0.016291,0.099600,NaN,0.116215,-0.017135,0.056239
YearBuilt,0.152935,0.061145,0.643484,-0.423477,1.000000,0.671317,0.319002,0.199108,-0.075099,0.198060,...,0.531544,0.256379,0.314380,-0.420435,0.032137,-0.098873,NaN,-0.094787,0.022852,-0.027824
YearRemodAdd,0.087849,0.005068,0.582143,-0.027233,0.671317,1.000000,0.189081,0.072475,-0.138022,0.236008,...,0.414228,0.214976,0.313056,-0.211015,0.030068,-0.049520,NaN,-0.077198,0.018674,0.011586
MasVnrArea,0.148737,0.089639,0.399781,-0.138321,0.319002,0.189081,1.000000,0.231383,-0.051427,0.083979,...,0.351112,0.185733,0.145842,-0.146518,-0.020003,-0.030240,NaN,-0.051143,0.024346,0.005247
BsmtFinSF1,0.175844,0.170325,0.171829,-0.008627,0.199108,0.072475,0.231383,1.000000,0.004386,-0.597790,...,0.247944,0.195621,0.055335,-0.107047,-0.029998,0.038937,NaN,-0.010647,-0.017104,0.041855
BsmtFinSF2,-0.023832,-0.010906,-0.142646,0.070226,-0.075099,-0.138022,-0.051427,0.004386,1.000000,-0.184254,...,-0.040229,0.048784,-0.093747,0.001388,-0.007647,0.032433,NaN,-0.015216,-0.009169,0.019651
BsmtUnfSF,0.151633,0.082340,0.321134,-0.151016,0.198060,0.236008,0.083979,-0.597790,-0.184254,1.000000,...,0.193311,0.011677,0.156445,-0.023616,0.052116,-0.006791,NaN,-0.068844,0.064025,-0.053250
